<a href="https://colab.research.google.com/github/RCarteri/gerai_receitas/blob/main/gerai_receitas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cansado de abrir a geladeira e não saber o que fazer? 🍲 🤔
Apresentamos o Gerador de Receitas, a solução para suas dúvidas culinárias!
Com nosso aplicativo você pode:

- Informar os ingredientes que você tem em casa (por texto ou fotos!) 📸
- Tirar fotos dos ingredientes na hora, direto pelo aplicativo ou anexar as imagens!
- Receber sugestões de receitas deliciosas e fáceis de preparar! 😋
- Explorar um mundo de sabores com combinações que você nunca imaginou! 🤯
- Receber um relatório nutricional e de benefícios da sua receita.

Chega de desperdício! 🚫 🥬 Transforme os ingredientes esquecidos em pratos incríveis com o Gerador de Receitas.

Baixe agora e libere o chef que existe em você!

Definição das principais varíavei do projeto

In [1]:
caminho_imagens = "/content/imagens"
modelo_img="gemini-pro-vision"
modelo_txt="gemini-pro"
gemini_api_secret_name = 'API_KEY'

Importação das bibliotecas

In [2]:
import PIL.Image
import os
import google.generativeai as genai
import time
from google.colab import userdata
from IPython.display import Markdown, display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode

Instalação da ferramenta do google ai

In [3]:
!pip install -q -U google-generativeai

Funções para tirar fotos com a câmera do seu computador

In [4]:
def get_foto(quality=0.8):
  timestamp = int(time.time())
  filename = f'{caminho_imagens}/ingredient_{timestamp}.jpg'
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capturar';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg/content', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

def tirar_foto():
  try:
    filename = get_foto()
    print(f'Salvo em {filename}')
    display(Image(filename))
  except Exception as err:
    print(str(err))

Principais funções do projeto

In [5]:
def get_api_key():
  try:
    GOOGLE_API_KEY=userdata.get(gemini_api_secret_name)
    genai.configure(api_key=GOOGLE_API_KEY)
  except userdata.SecretNotFoundError as e:
    print(f'Google ai api key não encontrada\n\nEspera-se que você crie uma secret chamada {gemini_api_secret_name} no Colab\n\nVisite https://makersuite.google.com/app/apikey para criar uma API key\n\nArmazene isso na seção de segredos no lado esquerdo do notebook (ícone de chave).\n\nNome da secret {gemini_api_secret_name}')
    raise e
  except userdata.NotebookAccessError as e:
    print(f'Você precisa conceder acesso a este notebook ao {gemini_api_secret_name} secret para que o notebook acesse a Gemini em seu nome.')
    raise e
  except Exception as e:
    print(f"Houve um erro desconhecido. Certifique-se de ter uma secret {gemini_api_secret_name} armazenado no Colab e é uma chave válida de https://makersuite.google.com/app/apikey")
    raise e

def gerar_modelo(modelo):
  return genai.GenerativeModel(model_name=modelo,
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)
def get_imagens():
  lista_arquivos = []
  for diretorio, _, arquivos in os.walk(caminho_imagens):
    for arquivo in arquivos:
      caminho_completo = os.path.join(diretorio, arquivo)
      lista_arquivos.append(caminho_completo)
  assert len(lista_arquivos) != 0, f"Nenhuma imagem encontrada no caminho: {caminho_imagens}"
  return lista_arquivos

def get_prompt_imagens():
  imagens_path = get_imagens()
  prompt_list = [msg_receita]
  for imagem_path in imagens_path:
    prompt_list.append(PIL.Image.open(imagem_path))
  return prompt_list

def print_markdown(msg):
  display(Markdown(msg))

def get_msg_escolha(escolha):
  if escolha == "1":
    return msg_digitar
  elif escolha == "2":
    return msg_anexar
  else:
    raise ValueError("Escolha inválida, digite 1 ou 2")

def anexar_imagens():
  opcao = input("\n")
  while opcao != "0":
    if opcao == "1":
      print_markdown(tutorial_anexar_imagem)
      opcao = input("\nAnexe todas as fotos e ao finalizar digite 0...\n")
    elif opcao == "2":
      tirar_foto()
      opcao = input("\nDigite 2 para tirar a próxima foto ou 0 para finalizar...\n")
    else:
      raise ValueError("Escolha inválida, digite 1, 2 ou 0")

def print_receita(escolha):
  prompt = [msg_receita]
  if escolha == "1":
    ingredientes = input("\n")
    prompt.append(ingredientes)
    modelo = modelo_txt
  elif escolha == "2":
    anexar_imagens()
    prompt = get_prompt_imagens()
    modelo = modelo_img
  print("\nGerando receita...\n")
  response = gerar_modelo(modelo).generate_content(prompt, stream=False)
  response.resolve()
  print_markdown(response.text)

def get_receita():
  print_markdown(msg_introducao)
  print("Digite 1 para informar os ingredientes ou 2 para anexar as imagens...\n")
  escolha = input()
  print_markdown(get_msg_escolha(escolha))
  print_receita(escolha)

Definição das configurações do gerador de texto do Google

In [6]:
get_api_key()

generation_config = {
  "candidate_count": 1,
  "temperature": 0,
}

safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

Definição das mensagens usadas

In [7]:
msg_introducao = """
  **Bem-vindo ao Chef Digital!** 👨‍🍳
  Pronto para descobrir receitas deliciosas com o que você tem em mãos?
  Basta escolher:
  * **Digitar ingredientes:** Conte-nos o que tem na sua geladeira e despensa.
  * **Anexar imagens:** Tire fotos dos seus ingredientes e deixe a mágica acontecer! ✨

  Qual será sua escolha hoje?\n\n
  """

msg_digitar = """\n
  Por favor, digite os ingredientes que você tem disponíveis, **separados por vírgula**.

  Por exemplo: **macarrão, tomate, queijo, manjericão**

  😊 Aguardo ansiosamente a sua lista!\n
  """

msg_anexar = """
  \n\n**Anexando fotos dos ingredientes:** 📸
  Você pode anexar fotos dos seus ingredientes de duas maneiras:

  1- Digite 1 para iniciar o processo de anexar as imagens.

  2- Digite 2 para tirar as fotos.

  😊 Aguardo ansiosamente a sua lista ou fotos dos ingredientes!\n
  """

tutorial_anexar_imagem = f"""
   - **Acesse o menu lateral esquerdo do Google Colab e clique no ícone de pasta.**
   - **Clique no botão "Upload" e selecione as fotos dos seus ingredientes.**
   - **Certifique-se de que as fotos sejam salvas na pasta `{caminho_imagens}`.**
   """

msg_receita = "Tendo como base os ingredientes a seguir, gere uma opção de receita em formato de tabela markdown com titulo, quantidade, ingredientes, fora da tabela o modo de preparo, beneficios e valor nutricional incluindo os números abreviados"

Chamada da função principal que cria a receita

(Às vezes o input não apareceu automaticamente 😅. Sem problemas! Por gentileza, poderia executar a célula novamente para que possamos continuar?)

In [8]:
get_receita()


  **Bem-vindo ao Chef Digital!** 👨‍🍳  
  Pronto para descobrir receitas deliciosas com o que você tem em mãos? 
  Basta escolher:
  * **Digitar ingredientes:** Conte-nos o que tem na sua geladeira e despensa.
  * **Anexar imagens:** Tire fotos dos seus ingredientes e deixe a mágica acontecer! ✨

  Qual será sua escolha hoje?


  

Digite 1 para informar os ingredientes ou 2 para anexar as imagens...

1




  Por favor, digite os ingredientes que você tem disponíveis, **separados por vírgula**. 

  Por exemplo: **macarrão, tomate, queijo, manjericão** 

  😊 Aguardo ansiosamente a sua lista!

  


massa, creme de leite, carne de gado, vinho, tomate, manjericao

Gerando receita...



**Receita de Ragu de Carne**

| Ingrediente | Quantidade |
|---|---|
| Massa | 500g |
| Creme de leite | 1 caixa (200g) |
| Carne de gado moída | 500g |
| Vinho tinto | 1/2 xícara |
| Tomate pelado | 1 lata (400g) |
| Manjericão fresco | 1/2 xícara |

**Modo de Preparo:**

1. Em uma panela grande, doure a carne moída em fogo médio.
2. Adicione o vinho e deixe evaporar.
3. Acrescente o tomate pelado, o manjericão e sal e pimenta a gosto.
4. Deixe cozinhar em fogo baixo por cerca de 1 hora, ou até que o molho engrosse.
5. Cozinhe a massa de acordo com as instruções da embalagem.
6. Misture o creme de leite ao molho e aqueça até ferver.
7. Sirva o ragu sobre a massa.

**Benefícios:**

* Rico em proteínas e ferro
* Fonte de vitaminas e minerais
* Auxilia na saciedade

**Valor Nutricional (por porção):**

* Calorias: 500
* Proteínas: 30g
* Carboidratos: 50g
* Gorduras: 20g